# MINIMIZE_SCALAR

## Overview
The `minimize_scalar` function finds the minimum value of a scalar mathematical function. This is useful for business users in Excel who need to optimize costs, maximize efficiency, or find the best value for a given scenario. The function leverages `scipy.optimize.minimize_scalar` and allows users to specify the function as a string, with optional bounds and method.

## Usage
To use the `minimize_scalar` function in Excel, enter it as a formula in a cell, specifying your function expression, optional bounds, and method:

```excel
=MINIMIZE_SCALAR(func_expr, bounds, method)
```

## Arguments
| Argument    | Type         | Required | Description                                                                 | Example |
|-------------|--------------|----------|-----------------------------------------------------------------------------|---------|
| func_expr   | string       | Yes      | The function to minimize, as a string (e.g., 'x**2 + 3*x + 2')              | 'x**2 + 3*x + 2' |
| bounds      | 2D list/None | No       | Optional. 2D list [[min, max]] for bounded minimization                     | [[0, 10]] |
| method      | string/None  | No       | Optional. Optimization method: 'brent', 'bounded', or 'golden'              | 'bounded' |

## Returns
| Return Value | Type         | Description                                                    | Example |
|--------------|-------------|----------------------------------------------------------------|---------|
| result       | 2D list     | [[x, fun]]: x-value of minimum and minimum value                | [[-1.5, -0.25]] |
| error        | string      | Error message if input is invalid or an error occurs            | "Error during minimization: ..." |

## Examples

### Minimize a Quadratic Cost Function
**Sample Input:**
- func_expr: 'x**2 + 3*x + 2'
- bounds: None
- method: None

**Sample Call:**
```excel
=MINIMIZE_SCALAR("x**2 + 3*x + 2", None, None)
```
**Sample Output:**
[[ -1.5, -0.25 ]]

### Minimize a Function with Bounds
**Sample Input:**
- func_expr: '(x-5)**2 + 10'
- bounds: [[0, 10]]
- method: 'bounded'

**Sample Call:**
```excel
=MINIMIZE_SCALAR("(x-5)**2 + 10", [[0, 10]], "bounded")
```
**Sample Output:**
[[ 5.0, 10.0 ]]

### Invalid Input
**Sample Input:**
- func_expr: 'x***2 + 3x + 2' (invalid)
- bounds: None
- method: None

**Sample Call:**
```excel
=MINIMIZE_SCALAR("x***2 + 3x + 2", None, None)
```
**Sample Output:**
"Error during minimization: ..."

## Limitations
- The function expression must be a valid Python expression in terms of x.
- If bounds are provided, method should be 'bounded'.
- Only scalar (single-variable) functions are supported.
- If input types are invalid or an error occurs, a string error message is returned instead of raising an exception.

In [13]:
from scipy.optimize import minimize_scalar as scipy_minimize_scalar
import math

def minimize_scalar(func_expr, bounds=None, method=None):
    """
    Minimizes a scalar function using scipy.optimize.minimize_scalar.
    Args:
        func_expr (str): A string representing the function to minimize, e.g., 'x**2 + 3*x + 2'.
        bounds (list, optional): A 2D list [[min, max]] specifying the bounds for bounded methods.
        method (str, optional): Optimization method: 'brent', 'bounded', or 'golden'.
    Returns:
        list: [[x, fun]] where x is the location of minimum and fun is the minimum value, or a string with an error message
    """
    if not (isinstance(func_expr, str)):
        return "func_expr must be a string."
    if bounds is not None:
        if not (isinstance(bounds, list) and all(isinstance(b, list) and len(b) == 2 for b in bounds)):
            return "bounds must be a 2D list (list of [min, max] lists) or None."
    if method is not None and not isinstance(method, str):
        return "method must be a string or None."
    def func(x):
        return eval(func_expr, {"x": x, "math": math})
    if 'x' not in func_expr:
        return "Function expression must contain the variable 'x'."
    kwargs = {}
    if bounds is not None:
        if isinstance(bounds, list) and len(bounds) == 1 and isinstance(bounds[0], list) and len(bounds[0]) == 2:
            min_val, max_val = bounds[0][0], bounds[0][1]
            kwargs['bounds'] = [min_val, max_val]
        else:
            kwargs['bounds'] = bounds
    if method is not None:
        kwargs['method'] = method
    try:
        result = scipy_minimize_scalar(func, **kwargs)
        return [[float(result.x), float(result.fun)]]
    except Exception as e:
        return f"Error during minimization: {str(e)}"

In [14]:
# Unit Tests
import ipytest
ipytest.autoconfig()

def test_demo_quadratic_unbounded():
    # All arguments set, including optional
    func_expr = "x**2 + 3*x + 2"
    bounds = None
    method = None
    result = minimize_scalar(func_expr, bounds, method)
    assert isinstance(result, list)
    assert len(result) == 1
    assert isinstance(result[0], list)
    assert len(result[0]) == 2
    assert all(isinstance(x, float) for x in result[0])

def test_demo_bounded_minimum():
    func_expr = "(x-5)**2 + 10"
    bounds = [[0, 10]]
    method = "bounded"
    result = minimize_scalar(func_expr, bounds, method)
    assert isinstance(result, list)
    assert len(result) == 1
    assert isinstance(result[0], list)
    assert len(result[0]) == 2
    assert all(isinstance(x, float) for x in result[0])
    # Should be close to [5.0, 10.0]
    assert abs(result[0][0] - 5.0) < 1e-3
    assert abs(result[0][1] - 10.0) < 1e-3

def test_demo_invalid_expression():
    func_expr = "x***2 + 3x + 2"
    bounds = None
    method = None
    result = minimize_scalar(func_expr, bounds, method)
    assert isinstance(result, str) and ("error" in result.lower() or "must be" in result.lower())

def test_demo_missing_x():
    func_expr = "5 + 7"
    bounds = None
    method = None
    result = minimize_scalar(func_expr, bounds, method)
    assert isinstance(result, str) and ("function expression must contain" in result.lower())

ipytest.run()

.......                                                                                         [100%]
4 passed in 0.04s
                                                                                         [100%]
4 passed in 0.04s


.......                                                                                         [100%]
4 passed in 0.04s
                                                                                         [100%]
4 passed in 0.04s


<ExitCode.OK: 0>

In [18]:
# Interactive Demo

import gradio as gr

examples = [
    [
        "x**2 + 3*x + 2",
        [[-1000, 1000]],
        "bounded"
    ],
    [
        "(x-5)**2 + 10",
        [[0, 10]],
        "bounded"
    ]
]

demo = gr.Interface(
    fn=minimize_scalar,
    inputs=[
        gr.Textbox(label="Function Expression", value="x**2 + 3*x + 2"),
        gr.Dataframe(headers=["min", "max"], label="Bounds (2D list, optional)", row_count=1, col_count=2, type="array", value=[[-1000, 1000]]),
        gr.Textbox(label="Method (optional)", value="bounded"),
    ],
    outputs=gr.Dataframe(headers=["x", "fun"], label="[x, minimum value] or error message"),
    examples=examples,
    description="Find the minimum of a scalar function. All arguments must be set for each example.",
    flagging_mode="never",
)
demo.launch()

* Running on local URL:  http://127.0.0.1:7883
* To create a public link, set `share=True` in `launch()`.
